Introduction



Import statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime as dt
import math

here = os.getcwd()

Load the 2020 data set and the reference files.

In [2]:
a = pd.read_csv("data/2020/2020data.csv")
b = pd.read_csv('data/2020/2020datasimp.csv')
c = pd.read_csv('data/2020/map-keys-2020.csv')
d = pd.read_csv("refs/florahelvetica/florahelveticarcsvnew.csv")

print(a.columns)
print(b.columns)
print(c.columns)
print(d.columns)

Index(['id', 'obs', 'place', 'date', 'time', 'sci', 'name', 'notes'], dtype='object')
Index(['place', 'date', 'time', 'sci', 'name'], dtype='object')
Index(['lon', 'lat', 'place'], dtype='object')
Index(['acceptedtaxonkey', 'acceptedscientificname', 'numberofoccurrences',
       'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species'],
      dtype='object')


In [3]:
c.rename(columns={'X':'lon', 'Y':'lat', 'Name':'place'}, inplace=True)
b.rename(columns={'sci':'species'}, inplace=True)

In [4]:
replacedict = {
    'verbanum bonariensis':'verbena bonariensis',
    'medicago varia':'medicago sativa',
    "oenothera":"oenothera biennis",
    "geranium pratens":"geranium pratense",
    "senecio jacobaea": "jacobaea vulgaris",
    "oenothera biennis ": "oenothera biennis",
    "solidalgo canadensis": "solidago canadensis",
    "verbascum lynchitis":"verbascum lychnitis",
    "verbascum negris":"verbascum nigrum",
    "securigea varia": "securigera varia",
    "melilotus officianalis": "melilotus officinalis",
    "knautia maxima": "knautia dipsacifolia",
    "hieracium aurantiacum":"pilosella aurantiaca",
    "sysimbrium officinale":"sisymbrium officinale",
    "geranium robertanium":"geranium robertianum",
    "mycelis muralis": "lactuca muralis",
    "calamintha nepeta":
    "clinopodium nepeta",
    "polygonum persicaria":"persicaria maculosa",
    "sorbus aria":"aria edulis"
}


In [5]:
def new_func(x,keys):
    try:
        data = keys[x]
    except:
        data = x
    return data
b["species1"] = b.species.map(lambda x: new_func(x, replacedict))

# b["species"] = b.species1

In [6]:
b['string_list'] = b.species1.str.split( " ")
b['string_list'] = b.string_list.map(lambda x: [name for name in x if name != 'agg.'])

In [7]:
b['string_list']

0      [plantago, lanceolata]
1          [centaurea, nigra]
2           [plantago, media]
3        [chenopodium, album]
4          [centaurea, jacea]
                ...          
932       [allium, carinatum]
933       [viburnum, lantana]
934    [filipendula, ulmaria]
935      [lythrum, salicaria]
936    [lysimachia, vulgaris]
Name: string_list, Length: 937, dtype: object

In [8]:
b['slug'] = b.string_list.map(lambda x: "-".join(x))
b['species'] = b.string_list.map(lambda x: " ". join(x))
b['place'] = b.place.map(lambda x:  x.replace(" ","-"))

In [9]:
def change_string(x):
    try:
        s_data = x.split('.')
        data = s_data[::-1]
        data = "-".join(data)
    except:
        print("no luck")
        data = x
    
    return data
b['new_date'] = b.date.map(lambda x: change_string(x))

In [10]:
def make_timestamp(x):
    try:        
        data = dt.datetime.strptime(x, "%Y-%m-%d")        
    except:        
        data = 'no date'
    
    return data
b['check_date'] = b.new_date.map(lambda x: make_timestamp(x))
b['date'] = b.new_date

print(F"\nThese are the records in b with no valid date:\n\n{b[b.check_date == 'no date']}\n")


These are the records in b with no valid date:

Empty DataFrame
Columns: [place, date, time, species, name, species1, string_list, slug, new_date, check_date]
Index: []



In [11]:
b_to_csv = b[['place','date','species', 'name', 'slug']].copy()
b_not_in_d = [x for x in b_to_csv.species.unique() if x not in d.species.unique()]
print(F"\nThese are the species in b that do not match d:\n\n{b_not_in_d }\n")


These are the species in b that do not match d:

['erigeron annuus', 'verbanum bonariensis ', 'taraxacum', 'rubus', 'rubus rubus sect.', 'tripleurospermum maritimum', 'persicaria longiseta', 'ligustrum vulgare', 'anemone sylvestris', 'x', 'oenothera biennis aggr.', 'packera aurea', 'dipsacus fullonum', 'rubus fruticosus', 'corylus cornuta', 'ligustrum sinense', 'eupatorium serotinum', 'stellaria aquatica', 'geranium pyrenaicum ', 'calamintha nepeta', 'solanum ptychanthum', 'salix nigra', 'phlox pilosa', 'hieracium morulum', 'sorbus aria']



In [12]:
b_to_csv

,place,date,species,name,slug
0,alleestrasse-1,2020-09-02,plantago lanceolata,plantain lancéolé,plantago-lanceolata
1,alleestrasse-1,2020-09-02,centaurea nigra,centaurée noire,centaurea-nigra
2,alleestrasse-1,2020-09-02,plantago media,plantain moyen,plantago-media
3,alleestrasse-1,2020-09-02,chenopodium album,chénopode blanc,chenopodium-album
4,alleestrasse-1,2020-09-02,centaurea jacea,centaurée jacée,centaurea-jacea
...,...,...,...,...,...
932,weissenau,2020-08-10,allium carinatum,ail caréné,allium-carinatum
933,weissenau,2020-08-10,viburnum lantana,"mancienne, viorne lantane",viburnum-lantana
934,weissenau,2020-08-10,filipendula ulmaria,"reine des prés, spirée",filipendula-ulmaria
935,weissenau,2020-08-10,lythrum salicaria,salicaire commune,lythrum-salicaria


In [13]:
break

SyntaxError: 'break' outside loop (<ipython-input-13-6aaf1f276005>, line 1)

In [14]:
c['place'] = c.place.str.split('\n').str[0]
c.place = c.place.str.lower()
c.place = c.place.map(lambda x:  x.replace(" ","-"))


In [16]:
# saving the location map keys
afilename= F"{here}/data/2020/map-keys-2020.csv"
c.to_csv(afilename, index=False)

In [17]:
afilename = here+"/data/2020/species2020_n23.csv"

minus_twenty_three = b_to_csv[~b_to_csv.species.isin(b_not_in_d)]

minus_twenty_three.to_csv(afilename, index=False)

In [ ]:
replacedict["verbanum bonariensis"]

In [ ]:
break

In [ ]:
# find place names that don't match:
a_locs = a.place.unique()
b_locs = b.place.unique()
c_locs = c.place.unique()
b_species = b.species.unique()
d_species = d.species.unique()

# get the symetric difference between the different place name lists
placenames = list(set(c_locs) | set(b_locs))
intersect_placenames = list(set(c_locs) & set(b_locs))
print(F"There are {len(placenames)} different place names\n")
print(F"\nThere are {len(b_locs)} different names in b\n")
print(F"\nThere are {len(c_locs)} different names in c\n")
print(F"\nThere are {len(intersect_placenames)} shared between b and c")

#find the species in the data set that are not in the flora helvetica set
speciesnames = list(set(d_species) | set(b_species))
intersect_speciesnames = list(set(d_species) & set(b_species))
in_b_not_d = list(set(b_species) - set(intersect_speciesnames))
print(F"There are {len(speciesnames)} different species names\n")
print(F"\nThere are {len(b_species)} different names in b\n")
print(F"\nThere are {len(d_species)} different names in d\n")
print(F"\nThere are {len(intersect_speciesnames)} shared between b and d")
print(F"\nThere are {len(in_b_not_d)} in b not in d")

In [ ]:
b['slug'] = b.species.map(lambda x: "-".join([x for x in x.split(" ")]))

In [ ]:
# saving the sample data




In [ ]:
b['a_new_name'] = b.species.map(lambda x: F"{x}three")

In [ ]:
#attempt to correct species names

#function fixes the scientific name in the 4th position of the 
y = b.species
for k in range(len(y[3])):
    if y[3][k] == "verbanum bonariensis":
        y[3][k] = "verbena bonariensis"
    if y[3][k] == "medicago varia":
        y[3][k] = "medicago sativa"
    if y[3][k] == "geranium pratens":
        y[3][k] = "geranium pratense"
    if y[3][k] == "oenothera":
        y[3][k] = "oenothera biennis"
    if y[3][k] == "senecio jacobaea":
        y[3][k] = "jacobaea vulgaris"
    if y[3][k] == "oenothera biennis ":
        y[3][k] = "oenothera biennis"
    if y[3][k] == "solidalgo canadensis":
        y[3][k] = "solidago canadensis"
    if y[3][k] == "verbascum lynchitis":
        y[3][k] = "verbascum lychnitis"
    if y[3][k] == "verbascum negris":
        y[3][k] = "verbascum nigrum"
    if y[3][k] == "securigea varia":
        y[3][k] = "securigera varia"
    if y[3][k] == "melilotus officianalis":
        y[3][k] = "melilotus officinalis"
    if y[3][k] == "knautia maxima":
        y[3][k] = "knautia dipsacifolia"
    if y[3][k] == "hieracium aurantiacum":
        y[3][k] = "pilosella aurantiaca"
    if y[3][k] == "sysimbrium officinale":
        y[3][k] = "sisymbrium officinale"
    if y[3][k] == "geranium robertanium":
        y[3][k] = "geranium robertianum"
    if y[3][k] == "mycelis muralis":
        y[3][k] = "lactuca muralis"
    if y[3][k] == "calamintha nepeta":
        y[3][k] = "clinopodium nepeta"
    if y[3][k] == "polygonum persicaria":
        y[3][k] = "persicaria maculosa"
    if y[3][k] == "sorbus aria":
        y[3][k] = "aria edulis"
b.species = y
new_b_species = b.species.unique()
new_d_species = d.species.unique()
new_speciesnames = list(set(new_d_species) | set(new_b_species))
new_intersect_speciesnames = list(set(new_d_species) & set(new_b_species))
new_in_b_not_d = list(set(new_b_species) - set(new_intersect_speciesnames))
print(F"There are {len(new_speciesnames)} different species names\n")
print(F"\nThere are {len(new_b_species)} different names in b\n")
print(F"\nThere are {len(new_d_species)} different names in d\n")
print(F"\nThere are {len(new_intersect_speciesnames)} shared between b and d")
print(F"\nThere are {len(new_in_b_not_d)} in b not in d")


Clean 2020 data set and reference files

In [ ]:
data2020 = utility.removelistreturns(data2020)
floralist = utility.removelistreturns(floralist)
data2020 = utility.makenestedlist(data2020)
floralist = utility.makenestedlist(floralist)
data2020 = utility.arrange2020data(data2020)
floralist = utility.arrangeflorahelveticadata(floralist)
data2020[3] = utility.removenameendings(data2020[3])
data2020 = utility.fixscinames(data2020)
data2020 = utility.fixdate(data2020)
data2020 = utility.fixtime(data2020)

Initial data analysis

In [ ]:
#total number of different values for each variable in data.
occurencesummary = utility.occurencelists(data2020)
print(occurencesummary[3])
newfile = open("occurencesummary.txt", 'w')
newfile.write(occurencesummary)
newfile.close()
comparison = utility.compared(occurencesummary[3][1], floralist[9])
print(comparison)